In [1]:
using OceananigansLagrangianFilter
using OceananigansLagrangianFilter: jld2_to_netcdf
using CairoMakie
using CUDA


Precompiling packages...
   4375.9 ms  ✓ OceananigansLagrangianFilter
  1 dependency successfully precompiled in 6 seconds. 356 already precompiled.


In [2]:
filter_config = OfflineFilterConfig(original_data_filename="SW_vort_with_tracer.jld2", 
                                    var_names_to_filter = ("ω", "T"), 
                                    velocity_names = ("u","v"),
                                    architecture = GPU(),
                                    Δt = 1e-3,
                                    T_out=0.1,
                                    N=2,
                                    freq_c = 1,
                                    output_netcdf = true,
                                    delete_intermediate_files = false)


Setting filter parameters to use Butterworth squared, order 4, cutoff frequency 1


OfflineFilterConfig("SW_vort_with_tracer.jld2", ("ω", "T"), ("u", "v"), 0.0, 40.0, 40.0, GPU{CUDABackend}(CUDABackend(false, true)), 0.1, (a1 = 0.09567085809127245, b1 = 0.23096988312782168, c1 = 0.3826834323650898, d1 = 0.9238795325112867, a2 = 0.23096988312782168, b2 = 0.09567085809127246, c2 = 0.9238795325112867, d2 = 0.38268343236508984, N_coeffs = 2), 0.001, InMemory{Int64}(1, 4), true, "forward_output.jld2", "backward_output.jld2", "combined_output.jld2", false, 5, false, true, WENO{3, Float64, Float32}(order=5)
├── buffer_scheme: WENO{2, Float64, Float32}(order=3)
└── advection_velocity_scheme: Centered(order=4), 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo
├── Periodic x ∈ [-1.91418e-18, 6.28319) regularly spaced with Δx=0.0245437
├── Periodic y ∈ [-1.91418e-18, 6.28319) regularly spaced with Δy=0.0245437
└── Flat z                               )

In [ ]:
run_offline_Lagrangian_filter(filter_config)

Loaded data from SW_vort_with_tracer.jld2
Created original variables: NamedTuple with 2 Fields on 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo:
├── T: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
└── ω: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
Created filtered variables: (:ω_C1, :ω_C2, :T_C1, :T_C2, :xi_u_C1, :xi_u_C2, :xi_v_C1, :xi_v_C2, :ω_S1, :ω_S2, :T_S1, :T_S2, :xi_u_S1, :xi_u_S2, :xi_v_S1, :xi_v_S2)
Created forcing for filtered variables 
Created model
Initialised filtered variables
Defined outputs
Defined simulation


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds


In [ ]:
# Animate
timeseries1 = FieldTimeSeries(filter_config.combined_output_filename, "ω")
timeseries2 = FieldTimeSeries(filter_config.combined_output_filename, "ω_filtered")
timeseries3 = FieldTimeSeries(filter_config.combined_output_filename, "ω_filtered_regrid")
times = timeseries1.times

set_theme!(Theme(fontsize = 20))
fig = Figure(size = (1000, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "y",
               limits = ((0, 2π), (0, 2π)),
               aspect = AxisAspect(1))

ax1 = Axis(fig[2, 1]; title = "Raw vorticity", axis_kwargs...)
ax2 = Axis(fig[2, 2]; title = "Filtered vorticity", axis_kwargs...)
ax3 = Axis(fig[2, 3]; title = "Filtered regridded vorticity", axis_kwargs...)


n = Observable(1)
Observable(1)

var1 = @lift timeseries1[$n]
var2 = @lift timeseries2[$n]
var3 = @lift timeseries3[$n]

heatmap!(ax1, var1; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax2, var2; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax3, var3; colormap = :balance, colorrange = (-1, 1))


title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:3], title, fontsize=24, tellwidth=false)

fig

frames = 1:length(times)

@info "Making an animation"

CairoMakie.record(fig, "LF_movie.mp4", frames, framerate=24) do i
    n[] = i
end

[ Info: Making an animation


"LF_movie.mp4"

In [14]:
jld2_to_netcdf(combined_output_filename, joinpath(@__DIR__, "combined_output.nc"))

closed Dataset

In [4]:
jld2_to_netcdf(filter_config.forward_output_filename, joinpath(@__DIR__, "forward_output.nc"))
jld2_to_netcdf(filter_config.backward_output_filename, joinpath(@__DIR__, "backward_output.nc"))

Wrote NetCDF file to /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/turbulent_SW/forward_output.nc
Wrote NetCDF file to /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/turbulent_SW/backward_output.nc
